<a href="https://colab.research.google.com/github/solcanalla/fiumark/blob/main/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelo:XGBoost#

Ensamble de tipo boosting.

##Inicialización##


In [15]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import seaborn as sns
from xgboost import XGBClassifier

In [26]:
!git clone https://github.com/solcanalla/fiumark.git
!git pull origin main
import fiumark.preprocessing as pp

Cloning into 'fiumark'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 126 (delta 69), reused 44 (delta 18), pack-reused 0
Receiving objects: 100% (126/126), 34.72 KiB | 683.00 KiB/s, done.
Resolving deltas: 100% (69/69), done.
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 3), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (8/8), done.
From https://github.com/solcanalla/fiumark
 * branch            main       -> FETCH_HEAD
   97ba6ff..26cf7ba  main       -> origin/main
Updating 97ba6ff..26cf7ba
error: The following untracked working tree files would be overwritten by merge:
	xgboost.csv
Please move or remove them before you merge.
Aborting


##Pre procesamiento##

In [17]:
pd.options.display.max_columns = None
df = pp.get_dataset()
df = pp.knn_preprocessing(df)
X_train, X_test, y_train, y_test = pp.get_train_test_data(df)

##Entrenamiento##

In [19]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': np.arange(50,100,10),
    'max_depth': np.arange(1, 10),
    'min_child_weight': np.arange(1, 3)
}

clf = XGBClassifier()

gscv = GridSearchCV(
    clf, params, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

In [20]:
print(f"Best score: {gscv.best_score_}")
print(f"Best params {gscv.best_params_}")

Best score: 0.8653038772069914
Best params {'max_depth': 2, 'min_child_weight': 2, 'n_estimators': 60}


##Evaluación del modelo##

In [21]:
y_pred = gscv.predict(X_test)

###Precision recall y F1-score###

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.78      0.96      0.86        70
         1.0       0.91      0.63      0.74        51

    accuracy                           0.82       121
   macro avg       0.85      0.79      0.80       121
weighted avg       0.84      0.82      0.81       121



###Accuracy###

In [23]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8181818181818182

###Auc-Roc###

In [24]:
from sklearn.metrics import roc_auc_score
auc_roc = roc_auc_score(y_test, y_pred)
auc_roc

0.792296918767507

##Predicción##

In [25]:
X_holdout = pd.read_csv('https://drive.google.com/uc?export=download&id=1I980-_K9iOucJO26SG5_M8RELOQ5VB6A')
X = pp.knn_preprocessing(X_holdout)
prediction = pd.DataFrame(gscv.predict(X),columns=['volveria'],dtype=int)
pred_formatted = pd.concat([X_holdout.id_usuario,prediction],axis=1)
pred_formatted.to_csv('xgboost.csv')
pred_formatted

,id_usuario,volveria
0,650,0
1,234,0
2,31,0
3,552,0
4,411,0
...,...,...
85,354,0
86,82,0
87,172,0
88,8,1
